**I recommend you run the first code cell of this notebook immediately, to start provisioning drake on the cloud machine, then you can leave this window open as you [read the textbook](http://manipulation.csail.mit.edu/clutter.html).**

# Notebook Setup

The following cell will:
- on Colab (only), install Drake to `/opt/drake`, install Drake's prerequisites via `apt`, and add pydrake to `sys.path`.  This will take approximately two minutes on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  If you navigate between notebooks using Colab's "File->Open" menu, then you can avoid provisioning a separate machine for each notebook.
- define some utility methods/classes that will eventually disappear from this notebook and live in drake.

You will need to rerun this cell if you restart the kernel, but it should be fast because the machine will already have drake installed.

In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

if 'google.colab' in sys.modules and importlib.util.find_spec('manipulation') is None:
    urlretrieve(f"http://manipulation.csail.mit.edu/scripts/setup/setup_manipulation_colab.py",
                "setup_manipulation_colab.py")
    from setup_manipulation_colab import setup_manipulation
    setup_manipulation(manipulation_sha='master', drake_version='latest', drake_build='continuous')

# Determine if this notebook is currently running as a notebook or a unit test.
from IPython import get_ipython
running_as_notebook = get_ipython() and hasattr(get_ipython(), 'kernel')

# Setup rendering (with xvfb), if necessary:
import os
if 'google.colab' in sys.modules and os.getenv("DISPLAY") is None:
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(1400, 900))
    display.start()

server_args = []
if 'google.colab' in sys.modules:
  server_args = ['--ngrok_http_tunnel']

# Start two meshcat servers (one for 2d) to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc_2d, zmq_url_2d, web_url_2d = start_zmq_server_as_subprocess(server_args=server_args)
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

# Imports
import numpy as np
from IPython.display import display, HTML
from ipywidgets import Textarea

from pydrake.all import ( 
    AddMultibodyPlantSceneGraph, ConnectMeshcatVisualizer, 
    DiagramBuilder, RigidTransform, RotationMatrix, Box,    
    CoulombFriction, FindResourceOrThrow, FixedOffsetFrame, 
    GeometryInstance, MeshcatContactVisualizer, Parser, PlanarJoint,  
    JointIndex, Simulator
)

import open3d as o3d
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import meshcat
import meshcat.geometry as g
import meshcat.transformations as tf

from pydrake.all import (
    ConnectPlanarSceneGraphVisualizer,
    ConnectDrakeVisualizer, DepthCameraProperties, RgbdSensor,
    RandomGenerator, UniformlyRandomRotationMatrix, RollPitchYaw,
    MakeRenderEngineVtk, RenderEngineVtkParams, Role, UnitInertia, set_log_level
)

from ipywidgets import Dropdown, FloatSlider, Layout
from pydrake.all import (
    Sphere, Cylinder, Box, Capsule, Ellipsoid, SpatialInertia)


#from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback
from manipulation.jupyter_widgets import MakeJointSlidersThatPublishOnCallback
from manipulation.meshcat_utils import draw_open3d_point_cloud, draw_points
from manipulation.open3d_utils import create_open3d_point_cloud
from manipulation.mustard_depth_camera_example import MustardExampleSystem
from manipulation.utils import FindResource

set_log_level("warn");

# Falling things (in 2D)


In [ ]:
def clutter_gen():
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)

    box = Box(10., 10., 10.)
    X_WBox = RigidTransform([0, 0, -5])
    mu = 1.0
    plant.RegisterCollisionGeometry(plant.world_body(), X_WBox, box, "ground", CoulombFriction(mu, mu))
    plant.RegisterVisualGeometry(plant.world_body(), X_WBox, box, "ground", [.9, .9, .9, 1.0])
    planar_joint_frame = plant.AddFrame(FixedOffsetFrame("planar_joint_frame", plant.world_frame(), RigidTransform(RotationMatrix.MakeXRotation(np.pi/2))))

    parser = Parser(plant)

    sdf = FindResourceOrThrow("drake/examples/manipulation_station/models/061_foam_brick.sdf")

    for i in range(20 if running_as_notebook else 2):
        instance = parser.AddModelFromFile(sdf, f"object{i}")
        plant.AddJoint(PlanarJoint(f"joint{i}", planar_joint_frame, plant.GetFrameByName("base_link", instance), damping=[0,0,0]))

    plant.Finalize()

    vis = ConnectPlanarSceneGraphVisualizer(
        builder, 
        scene_graph,
        xlim=[-.6, .6],
        ylim=[-.1, 0.5],
        show=False,
    )

    diagram = builder.Build()
    simulator = Simulator(diagram)
    plant_context = plant.GetMyContextFromRoot(simulator.get_mutable_context())

    rs = np.random.RandomState()
    z = 0.1
    for i in range(plant.num_joints()):
        joint = plant.get_joint(JointIndex(i))
        joint.set_pose(plant_context, [rs.uniform(-.4, .4), z], rs.uniform(-np.pi/2.0, np.pi/2.0))
        z += 0.1

    vis.start_recording()
    simulator.AdvanceTo(1.5 if running_as_notebook else 0.1)
    vis.stop_recording()
    ani = vis.get_recording_as_animation(repeat=False)
    display(HTML(ani.to_jshtml()))
    
clutter_gen()

# Falling things (in 3D)


In [ ]:
def clutter_gen():
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)

    parser = Parser(plant)

    parser.AddModelFromFile(FindResourceOrThrow(
        "drake/examples/manipulation_station/models/bin.sdf"))
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("bin_base"))

    ycb = [("cracker", "003_cracker_box.sdf"), 
           ("sugar", "004_sugar_box.sdf"), 
           ("soup", "005_tomato_soup_can.sdf"), 
           ("mustard", "006_mustard_bottle.sdf"), 
           ("gelatin", "009_gelatin_box.sdf"), 
           ("meat", "010_potted_meat_can.sdf")]

    rs = np.random.RandomState()  # this is for python
    generator = RandomGenerator(rs.randint(1000))  # this is for c++
    for i in range(10 if running_as_notebook else 2):
        object_num = rs.randint(len(ycb))
        sdf = FindResourceOrThrow("drake/manipulation/models/ycb/sdf/" + ycb[object_num][1])
        link_name = "base_link_" + ycb[object_num][0]
        instance = parser.AddModelFromFile(sdf, f"object{i}")

    plant.Finalize()

    renderer = "my_renderer"
    scene_graph.AddRenderer(
        renderer, MakeRenderEngineVtk(RenderEngineVtkParams()))
    properties = DepthCameraProperties(width=640,
                                       height=480,
                                       fov_y=np.pi / 4.0,
                                       renderer_name=renderer,
                                       z_near=0.1,
                                       z_far=10.0)
    camera = builder.AddSystem(
        RgbdSensor(parent_id=scene_graph.world_frame_id(),
                   X_PB=RigidTransform(
                       RollPitchYaw(np.pi, 0, np.pi/2.0),
                       [0, 0, .8]),
                   properties=properties,
                   show_window=False))
    camera.set_name("rgbd_sensor")
    builder.Connect(scene_graph.get_query_output_port(),
                    camera.query_object_input_port())
    builder.ExportOutput(camera.color_image_output_port(), "color_image")

    # Note: if you're running this on a local machine, then you can 
    # use drake_visualizer to see the simulation.  (It's too slow to 
    # show the meshes on meshcat).
    vis = ConnectDrakeVisualizer(
        builder, 
        scene_graph
    )

    diagram = builder.Build()
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    plant_context = plant.GetMyContextFromRoot(context)

    z = 0.1
    for body_index in plant.GetFloatingBaseBodies():
        tf = RigidTransform(
                UniformlyRandomRotationMatrix(generator),  
                [rs.uniform(-.15,.15), rs.uniform(-.2, .2), z])
        plant.SetFreeBodyPose(plant_context, 
                              plant.get_body(body_index),
                              tf)
        z += 0.1

    simulator.AdvanceTo(1.0 if running_as_notebook else 0.1)
    color_image = diagram.GetOutputPort("color_image").Eval(context)
    plt.figure()
    plt.imshow(color_image.data)
    plt.axis('off')

clutter_gen()

# Contact force "inspector"


In [ ]:
def contact_force_inspector(slope=0.0, mu=1.0, second_brick=False):
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.01)

    box = Box(10., 10., 10.)
    X_WBox = RigidTransform(RotationMatrix.MakeYRotation(slope), [0, 0, -5.05])
    plant.RegisterCollisionGeometry(plant.world_body(), X_WBox, box, "ground", CoulombFriction(mu, mu))
    plant.RegisterVisualGeometry(plant.world_body(), X_WBox, box, "ground", [.9, .9, .9, 1.0])

    parser = Parser(plant)
    brick_sdf = FindResource("models/planar_foam_brick_collision_as_visual.sdf")
    parser.AddModelFromFile(brick_sdf)
    frame = plant.AddFrame(FixedOffsetFrame("planar_joint_frame", plant.world_frame(), RigidTransform(RotationMatrix.MakeXRotation(np.pi/2))))
    plant.AddJoint(PlanarJoint("brick", frame, plant.GetFrameByName("base_link"), damping=[0,0,0]))

    if second_brick:
        brick2 = parser.AddModelFromFile(brick_sdf, "brick2")
        plant.AddJoint(PlanarJoint("brick2", frame, plant.GetFrameByName("base_link", brick2), damping=[0,0,0]))

    plant.Finalize()

    meshcat = ConnectMeshcatVisualizer(
        builder, 
        scene_graph, 
        zmq_url=zmq_url_2d,
        server_args=server_args)
    meshcat.set_planar_viewpoint(xmin=-.2, xmax=.2, ymin=-.2, ymax=0.3)

    contact_visualizer = builder.AddSystem(MeshcatContactVisualizer(meshcat, 
                                           force_threshold=0.0,
                                           contact_force_scale=1.0, 
                                           contact_force_radius=0.002,   
                                           plant=plant))
    builder.Connect(scene_graph.get_pose_bundle_output_port(),
                    contact_visualizer.GetInputPort("pose_bundle"))
    builder.Connect(plant.get_contact_results_output_port(),
                    contact_visualizer.GetInputPort("contact_results"))

    diagram = builder.Build()
    meshcat.load()
    context = diagram.CreateDefaultContext()

    textoutput = Textarea(value="", description="contact info: ", layout={'width':'800px','height':'100px'}, style={'description_width':'initial'})
    display(textoutput)
    def my_callback(plant_context):
        results = plant.get_contact_results_output_port().Eval(plant_context)
        textoutput.value = "" if results.num_point_pair_contacts()>0 else "no contact"
        for i in range(results.num_point_pair_contacts()):
            info = results.point_pair_contact_info(i)
            pair = info.point_pair()
            textoutput.value += f"slip speed:{info.slip_speed():.4f}, depth:{pair.depth:.4f}, "
            textoutput.value += "force:" + np.array2string(info.contact_force(), formatter={'float': lambda x: "{:5.2f}".format(x)}) + "\n"

    lower_limit = [-0.1, -0.1, -np.pi/2.0]
    upper_limit = [0.1, 0.1, np.pi/2.0]
    if second_brick:
        lower_limit += lower_limit
        upper_limit += upper_limit
        plant.SetPositions(
            plant.GetMyContextFromRoot(context),
            [0, 0, 0, 0.07, 0.07, 0.0])

    MakeJointSlidersThatPublishOnCallback(
        plant, diagram, context, my_callback, resolution=0.001,
        lower_limit=lower_limit, upper_limit=upper_limit)    

contact_force_inspector(
    slope=0.1, 
    mu=0.5,
    second_brick=True);

# Contact results "inspector"

This simple visualization shows some of the complexity of the contact geometry problem.  I will make it better, but right now, when you move the objects into contact of each other you will see three points:  the contact point is in **red**, the contact normal is added to the contact point with the tip as **green**, and the (scaled) contact force tip is drawn in **blue**.  Contact points on the bodies are drawn in **orange**.

In [ ]:
shapes = ["Point", "Sphere", "Cylinder", "Box", "Capsule", "Ellipsoid"]
shapeA = Dropdown(options=shapes,
                  description="Body A",
                  value="Box",
                  style = {'description_width': 'initial'})
shapeB = Dropdown(options=shapes,
                  description="Body B",
                  style = {'description_width': 'initial'})
display(shapeA)
display(shapeB)

def get_shape(shape):
    if shape == "Point":
        return Sphere(0.01)
    elif shape == "Sphere":
        return Sphere(1.0)
    elif shape == "Cylinder":
        return Cylinder(1.0, 2.0)
    elif shape == "Box":
        return Box(1.0, 2.0, 3.0)
    elif shape == "Capsule":
        return Capsule(1.0, 2.0)
    elif shape == "Ellipsoid":
        return Ellipsoid(1.0, 2.0, 3.0) 
    raise Exception("Unknown shape")


In [ ]:
def contact_inspector(shapeA, shapeB):
    builder = DiagramBuilder()

    shapeA = get_shape(shapeA.value)
    shapeB = get_shape(shapeB.value)

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
    frame = plant.AddFrame(FixedOffsetFrame("planar_joint_frame", plant.world_frame(), RigidTransform(RotationMatrix.MakeXRotation(np.pi/2))))

    mu = 0.5
    bodyA = plant.AddRigidBody("A", SpatialInertia(mass=1.0, p_PScm_E=np.array([0., 0., 0.]),
            G_SP_E=UnitInertia(1.0, 1.0, 1.0)))
    plant.RegisterCollisionGeometry(bodyA, RigidTransform(), shapeA, "A", CoulombFriction(mu, mu))
    plant.RegisterVisualGeometry(bodyA, RigidTransform(), shapeA, "A", [.9, .5, .5, 0.5])
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("A"))

    bodyB = plant.AddRigidBody("B", SpatialInertia(mass=1.0, p_PScm_E=np.array([0., 0., 0.]),
            G_SP_E=UnitInertia(1.0, 1.0, 1.0)))
    plant.RegisterCollisionGeometry(bodyB, RigidTransform(), shapeB, "B", CoulombFriction(mu, mu))
    plant.RegisterVisualGeometry(bodyB, RigidTransform(), shapeB, "B", [.5, .5, .9, 0.9])
    plant.AddJoint(PlanarJoint("B", frame, plant.GetFrameByName("B"), damping=[0,0,0]))

    plant.Finalize()

    meshcat = ConnectMeshcatVisualizer(
        builder, 
        scene_graph, 
        zmq_url=zmq_url_2d,
        server_args=server_args)
    meshcat.set_planar_viewpoint(xmin=-3.0, xmax=3.0, ymin=-.2, ymax=3.0)

#    contact_visualizer = builder.AddSystem(MeshcatContactVisualizer(meshcat, 
#                                           force_threshold=0.0,
#                                           contact_force_scale=1.0, 
#                                           contact_force_radius=0.002,   
#                                           plant=plant))
#    builder.Connect(scene_graph.get_pose_bundle_output_port(),
#                    contact_visualizer.GetInputPort("pose_bundle"))
#    builder.Connect(plant.get_contact_results_output_port(),
#                    contact_visualizer.GetInputPort("contact_results"))

    diagram = builder.Build()
    meshcat.load()
    context = diagram.CreateDefaultContext()

    contact = meshcat.vis["contact"]
    red = g.MeshLambertMaterial(color=0xff0000)
    green = g.MeshLambertMaterial(color=0x00ff00)
    blue = g.MeshLambertMaterial(color=0x0000ff)
    orange = g.MeshLambertMaterial(color=0xffa500)    

    textoutput = Textarea(value="", description="contact info: ", layout={'width':'800px','height':'100px'}, style={'description_width':'initial'})
    display(textoutput)
    def my_callback(plant_context):
        results = plant.get_contact_results_output_port().Eval(plant_context)
        contact.delete()
        textoutput.value = "" if results.num_point_pair_contacts()>0 else "no contact"
        for i in range(results.num_point_pair_contacts()):
            info = results.point_pair_contact_info(i)
            pair = info.point_pair()
            contact[str(i)].set_object(g.Sphere(0.02), red)
            contact[str(i)].set_transform(tf.translation_matrix(info.contact_point()))
            contact[str(i)+"A"].set_object(g.Sphere(0.01), orange)
            contact[str(i)+"A"].set_transform(tf.translation_matrix(pair.p_WCa))
            contact[str(i)+"B"].set_object(g.Sphere(0.01), orange)
            contact[str(i)+"B"].set_transform(tf.translation_matrix(pair.p_WCb))
            contact[str(i)+"normal"].set_object(g.Sphere(0.02), green)
            contact[str(i)+"normal"].set_transform(tf.translation_matrix(info.contact_point()-pair.nhat_BA_W ))
            contact[str(i)+"force"].set_object(g.Sphere(0.02), blue)
            contact[str(i)+"force"].set_transform(tf.translation_matrix(info.contact_point()+info.contact_force()/5000.0 ))
            textoutput.value += f"slip speed:{info.slip_speed():.4f}, separation_speed:{info.separation_speed():.4f}, depth:{pair.depth:.4f},\n"
            textoutput.value += "  point:" + np.array2string(info.contact_point(), formatter={'float': lambda x: "{:5.2f}".format(x)}) + "\n"
            textoutput.value += "  normal:" + np.array2string(-pair.nhat_BA_W, formatter={'float': lambda x: "{:5.2f}".format(x)}) + "\n"
            textoutput.value += "  force:" + np.array2string(info.contact_force(), formatter={'float': lambda x: "{:5.2f}".format(x)}) + "\n"


    lower_limit = [-3, -3, -np.pi/2.0]
    upper_limit = [3, 3, np.pi/2.0]
#    lower_limit += lower_limit
#    upper_limit += upper_limit
    plant.SetPositions(
        plant.GetMyContextFromRoot(context),
#        [0, 0, 0] +
        [1.2, 1.2, 0.0],
    )

    MakeJointSlidersThatPublishOnCallback(
        plant, diagram, context, my_callback, 
        lower_limit=lower_limit, upper_limit=upper_limit)    

contact_inspector(shapeA, shapeB);

# Contact geometry of the foam brick

In [ ]:
def show_brick_contact():
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.01)
    Parser(plant).AddModelFromFile(FindResource("models/061_foam_brick.sdf"))
    plant.Finalize()

    meshcat = ConnectMeshcatVisualizer(
        builder, 
        scene_graph, 
        zmq_url=zmq_url,
        server_args=server_args)

    diagram = builder.Build()
    meshcat.load()

show_brick_contact()

# Point cloud processing with Open3d

I've produced a scene with multiple cameras looking at our favorite YCB mustard bottle.  I've taken the individual point clouds, converted them into Open3d's point cloud format, estimated their normals, merged the point clouds, cropped then point clouds (to get rid of the geometry from the other cameras), then downsampled the point clouds.  (The order is important!)

I've pushed all of the point clouds to meshcat, but with many of them set to not be visible by default.  Use the drop-down menu to turn them on and off, and make sure you understand basically what is happening on each of the steps.

In [ ]:
def point_cloud_processing_example():
    # This just sets up our mustard bottle with three depth cameras positioned around it.
    system = MustardExampleSystem()

    plant = system.GetSubsystemByName("plant")

    # Evaluate the camera output ports to get the images.
    context = system.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    v = meshcat.Visualizer(zmq_url=zmq_url)
    v.delete()
    v["/Background"].set_property("visible", False)

    pcd = []
    for i in range(3):
        point_cloud = system.GetOutputPort(f"camera{i}_point_cloud").Eval(context)
        pcd.append(create_open3d_point_cloud(point_cloud))

        pcd[i].estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(
            radius=0.1, max_nn=30))

        camera = plant.GetModelInstanceByName(f"camera{i}")
        body = plant.GetBodyByName("base", camera)
        X_C = plant.EvalBodyPoseInWorld(plant_context, body)
        pcd[i].orient_normals_towards_camera_location(X_C.translation())
        draw_open3d_point_cloud(v[f"pointcloud{i}"], pcd[i])
        v[f"pointcloud{i}"].set_property("visible", False)

    # Merge point clouds.  (Note: You might need something more clever here for
    # noisier point clouds; but this can often work!)
    merged_pcd = pcd[0] + pcd[1] + pcd[2]
    draw_open3d_point_cloud(v["merged"], merged_pcd)
    v["merged"].set_property("visible", False)

    # Crop to region of interest.
    cropped_pcd = merged_pcd.crop(
        o3d.geometry.AxisAlignedBoundingBox(min_bound=[-.3, -.3, -.3],
                                            max_bound=[.3, .3, .3]))
    draw_open3d_point_cloud(v["cropped"], cropped_pcd)

    # Voxelize down-sample.  (Note that the normls still look reasonable)
    down_sampled_pcd = cropped_pcd.voxel_down_sample(voxel_size=0.005)
    draw_open3d_point_cloud(v["down_sampled"], down_sampled_pcd, normals_scale=0.01)
    # Let the normals be drawn, only turn off the object...
    v["down_sampled"]["<object>"].set_property("visible", False)

    # If we wanted to show it in the open3d visualizer, we would use...
    # print("Use 'n' to show normals, and '+/-' to change their size.")
    # o3d.visualization.draw_geometries([down_sampled_pcd])


point_cloud_processing_example()

In [ ]:
def get_mustard_point_cloud():
    """A "no frills" version of the example above, that returns the down-sampled point cloud"""
    system = MustardExampleSystem()

    plant = system.GetSubsystemByName("plant")

    # Evaluate the camera output ports to get the images.
    context = system.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    pcd = []
    for i in range(3):
        point_cloud = system.GetOutputPort(f"camera{i}_point_cloud").Eval(context)
        pcd.append(create_open3d_point_cloud(point_cloud))

        pcd[i].estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(
            radius=0.1, max_nn=30))

        camera = plant.GetModelInstanceByName(f"camera{i}")
        body = plant.GetBodyByName("base", camera)
        X_C = plant.EvalBodyPoseInWorld(plant_context, body)
        pcd[i].orient_normals_towards_camera_location(X_C.translation())

    # Merge point clouds.  (Note: You might need something more clever here for
    # noisier point clouds; but this can often work!)
    pcd = pcd[0] + pcd[1] + pcd[2]

    # Crop to region of interest.
    pcd = pcd.crop(
        o3d.geometry.AxisAlignedBoundingBox(min_bound=[-.3, -.3, -.3],
                                            max_bound=[.3, .3, .3]))    

    # Voxelize down-sample.  (Note that the normals still look reasonable)
    return pcd.voxel_down_sample(voxel_size=0.005)


# Estimating normals (and local curvature)

In [ ]:
from ipywidgets import IntSlider

def normal_estimation():
    system = MustardExampleSystem()
    context = system.CreateDefaultContext()

    v = meshcat.Visualizer(zmq_url=zmq_url)
    v.delete()
    v["/Background"].set_property("visible", False)

    point_cloud = system.GetOutputPort("camera0_point_cloud").Eval(context)
    pcd = create_open3d_point_cloud(point_cloud)
    draw_open3d_point_cloud(v["point_cloud"], pcd)

    kdtree = o3d.geometry.KDTreeFlann(pcd)
    pts = np.asarray(pcd.points)

    def my_callback(change):
        query = pts[change.new,:]
        draw_points(v["query"], query, [0.0, 1.0, 0.0], size=0.002)
        (num, indices, distances) = kdtree.search_hybrid_vector_3d(
            query=query, radius=0.1, max_nn=40)

        neighbors = pts[indices, :]
        draw_points(v["neighbors"], neighbors.T, [0.0, 0.0, 1.0], size=0.001)

        pstar = np.mean(neighbors,axis=0)
        prel = neighbors - pstar
        W = np.matmul(prel.T, prel)
        w, V = np.linalg.eigh(W)
        v["least_squares_basis"].set_object(g.triad(0.01))
        v["least_squares_basis"].set_transform(RigidTransform(RotationMatrix(np.fliplr(V)), query).GetAsMatrix4())

    slider = IntSlider(min=0, max=pts.shape[0])
    slider.observe(my_callback, names='value')
    slider.value=4165  # force callback immediately 
    display(slider)

normal_estimation()

# Generating grasp candidates

In [ ]:
class CollisionModel:

    def __init__(self, include_bin=False, gripper_grasp_area=False):
        builder = DiagramBuilder()
        self.plant, self.scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
        parser = Parser(self.plant)
        if include_bin:
            parser.AddModelFromFile(FindResourceOrThrow(
            "drake/examples/manipulation_station/models/bin.sdf"))
            self.plant.WeldFrames(self.plant.world_frame(), self.plant.GetFrameByName("bin_base"), RigidTransform([0,0,-0.015]))
        if gripper_grasp_area:
            self.gripper = self.plant.AddRigidBody("body",SpatialInertia())
            # Collision geometry is a box between the (open) finger tips.
            self.X_GF = RigidTransform([0, 0.10625, 0])
            box = Box(0.1, 0.012, 0.012)
            self.plant.RegisterCollisionGeometry(self.gripper, self.X_GF, box, "grasp_area", CoulombFriction(0.0, 0.0))
        else:
            gripper = parser.AddModelFromFile(FindResourceOrThrow(
                "drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf"), "gripper")
        self.plant.Finalize()
        self.diagram = builder.Build()
        self.context = self.diagram.CreateDefaultContext()
        self.plant_context = self.plant.GetMyContextFromRoot(self.context)
        self.scene_graph_context = self.scene_graph.GetMyContextFromRoot(self.context)
        if not gripper_grasp_area:
            self.plant.GetJointByName("left_finger_sliding_joint", gripper).set_translation(self.plant_context, -0.054)
            self.plant.GetJointByName("right_finger_sliding_joint", gripper).set_translation(self.plant_context, 0.054)
        self.gripper = self.plant.GetBodyByName("body")


class SimpleCollisionChecker:
    """ Note: Drake is threadsafe, if we want to parallelize these.  Just use one Context per thread. """

    def __init__(self):
        # Three different collision models
        self.hand = CollisionModel()
        self.grasp_area = CollisionModel(gripper_grasp_area=True)
        self.hand_and_bin = CollisionModel(include_bin=True)

    def HandCloudCollisions(self, X_G, cloud, threshold=0.0):
        self.hand.plant.SetFreeBodyPose(self.hand.plant_context, self.hand.gripper, X_G)
        query_object = self.hand.scene_graph.get_query_output_port().Eval(self.hand.scene_graph_context)
        # Note: I would like to have a faster interface for this.  See drake #9138.
        for pt in cloud.points:
            distances = query_object.ComputeSignedDistanceToPoint(pt, threshold=threshold)
            if distances:
                return True
        return False            

    def HandBinCollisions(self, X_G):
        self.hand_and_bin.plant.SetFreeBodyPose(self.hand_and_bin.plant_context, self.hand_and_bin.gripper, X_G)
        query_object = self.hand_and_bin.scene_graph.get_query_output_port().Eval(self.hand_and_bin.scene_graph_context)
        return query_object.HasCollisions()

    def GetPointsInGraspArea(self, X_G, cloud):
        # Also puts them into the frame of the finger grasp area.
        self.grasp_area.plant.SetFreeBodyPose(self.grasp_area.plant_context, self.grasp_area.gripper, X_G)
        query_object = self.grasp_area.scene_graph.get_query_output_port().Eval(self.grasp_area.scene_graph_context)
        X_F = X_G.multiply(self.grasp_area.X_GF)
        pts = []
        for pt in cloud.points:
            distances = query_object.ComputeSignedDistanceToPoint(pt, threshold=0.0)
            if distances:
                pts.append(X_F.multiply(pt))
        return np.asarray(pts)

def score_grasp_candidate(X_G, cloud_in_grasp_area):
#    v = meshcat.Visualizer(zmq_url=zmq_url)
#    draw_open3d_point_cloud(v["cloud_in_grasp_area"], cloud_in_grasp_area, normals_scale=0.01)
#    v["cloud_in_grasp_area"].set_transform(tf.translation_matrix([1., 0., 0]))
    print(cloud_in_grasp_area)
    return 1.0

def generate_grasp_candidate_antipodal(cloud, checker, rng, min_roll=-np.pi/3.0, max_roll=np.pi/3.0, meshcat=None):
    """
    Picks a random point in the cloud, and aligns the robot finger with the normal of that pixel. 
    The rotation around the normal axis is drawn from a uniform distribution over [min_roll, max_roll].
    """
    index = rng.integers(0,len(cloud.points)-1)

    # Use S for sample point/frame.
    p_WS = np.asarray(cloud.points[index])
    n_WS = np.asarray(cloud.normals[index])

    if meshcat:
        vertices = np.empty((3,2))
        vertices[:, 0] = p_WS
        vertices[:, 1] = p_WS + 0.05*n_WS
        meshcat.set_object(g.LineSegments(g.PointsGeometry(vertices),
                            g.MeshBasicMaterial(color=0xff0000)))

    assert np.isclose(np.linalg.norm(n_WS), 1.0)

    Gx = n_WS # gripper x axis aligns with normal
    # make orthonormal y axis, aligned with world down
    y = np.array([0.0, 0.0, -1.0])
    if np.abs(np.dot(y,Gx)) < 1e-6:
        # normal was pointing straight down.  reject this sample.
        return None

    Gy = y - np.dot(y,Gx)*Gx
    Gz = np.cross(Gx, Gy)
    R_WG = RotationMatrix(np.vstack((Gx, Gy, Gz)).T)
    p_GS_G = [0.045, 0.10625, 0]

    # Try orientations from the center out
    alpha = np.array([0.5, 0.65, 0.35, 0.8, 0.2, 1.0, 0.0])
    for theta in (min_roll + (max_roll - min_roll)*alpha):
        # Rotate the object in the hand by a random rotation (around the normal).
        R_WG2 = R_WG.multiply(RotationMatrix.MakeXRotation(theta))

        # Use G for gripper frame.
        p_SG_W = - R_WG2.multiply(p_GS_G)
        p_WG = p_WS + p_SG_W 

        X_G = RigidTransform(R_WG2, p_WG)
        if not checker.HandCloudCollisions(X_G, cloud) and not checker.HandBinCollisions(X_G):
            score = score_grasp_candidate(X_G, checker.GetPointsInGraspArea(X_G, cloud))
            return X_G, score
        #draw_grasp_candidate(X_G, f"collision/{theta:.1f}")

    return None, np.inf

def draw_grasp_candidate(X_G, prefix='gripper'):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    parser = Parser(plant)
    parser.AddModelFromFile(FindResourceOrThrow(
        "drake/examples/manipulation_station/models/bin.sdf"))
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("bin_base"), RigidTransform([0,0,-0.015]))
    gripper = parser.AddModelFromFile(FindResourceOrThrow(
        "drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf"), "gripper")
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("body"), X_G)
    plant.Finalize()
    
    frames_to_draw = {"gripper": {"body"}}
    meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, prefix=prefix, delete_prefix_on_load=False, frames_to_draw=frames_to_draw)
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    plant.GetJointByName("left_finger_sliding_joint", gripper).set_translation(plant_context, -0.054)
    plant.GetJointByName("right_finger_sliding_joint", gripper).set_translation(plant_context, 0.054)

    meshcat.load()
    diagram.Publish(context)

v = meshcat.Visualizer(zmq_url=zmq_url)
v.delete()
rng = np.random.default_rng()

collision_checker = SimpleCollisionChecker()

cloud = get_mustard_point_cloud()
draw_open3d_point_cloud(v["point cloud"], cloud)
X_G, score = generate_grasp_candidate_antipodal(cloud, collision_checker, rng, meshcat=v["candidate_debug"])
if X_G:
    print(f"score: {score:.2f}")
    draw_grasp_candidate(X_G)
else:
    print("failed to find grasp at this sample")


# Model directives

TODO(russt): Move this up to chapter 2?

In [ ]:
import pydrake.multibody.parsing

def draw_model(filename):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    parser = Parser(plant)
    parser.package_map().PopulateFromFolder(FindResource(""))
    pydrake.multibody.parsing.ProcessModelDirectives(
        pydrake.multibody.parsing.LoadModelDirectives(filename), plant, parser)
    plant.Finalize()
    
    meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url)
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()

    meshcat.vis.delete()
    meshcat.load()
    diagram.Publish(context)
    lower_limit = [-np.pi, -np.pi/4., -np.pi/4., -1, -1, 0]
    upper_limit = [0, np.pi/4., np.pi/4., 1, 1, 1]
    sliders = MakeJointSlidersThatPublishOnCallback(plant, diagram, context, lower_limit=lower_limit, upper_limit=upper_limit)
    sliders[0].value = -np.pi/2.0
    sliders[4].value = -.5
    sliders[5].value = 0.35

#draw_model(FindResource("models/mustard_w_cameras.yaml"))
draw_model(FindResource("models/clutter0.yaml"))
